<a href="https://colab.research.google.com/github/OmNai12/CSCI---266---Adv-Database-System/blob/main/Assingnment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**### Installing the required dependecies**

In [3]:
!pip install psycopg2-binary pandas python-dotenv

In [5]:
import os, psycopg2, pandas as pd
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [18]:
conn = psycopg2.connect(
    dbname=os.environ["PGDATABASE"],
    user=os.environ["PGUSER"],
    password=os.environ["PGPASSWORD"],
    host=os.environ["PGHOST"],
    port="5432",
    sslmode="require"
)

cur = conn.cursor()

### **Creating table and inserting data**

In [19]:
# 1. Create tables
cur.execute("""
DROP TABLE IF EXISTS Outcomes;
DROP TABLE IF EXISTS Ships;
DROP TABLE IF EXISTS Battles;
DROP TABLE IF EXISTS Classes;

CREATE TABLE Classes (
    class VARCHAR(50) PRIMARY KEY,
    type VARCHAR(20),
    country VARCHAR(50),
    numGuns INT,
    bore INT,
    displacement INT
);

CREATE TABLE Ships (
    name VARCHAR(50) PRIMARY KEY,
    class VARCHAR(50) REFERENCES Classes(class),
    launched INT
);

CREATE TABLE Battles (
    name VARCHAR(100) PRIMARY KEY,
    date VARCHAR(20)
);

CREATE TABLE Outcomes (
    ship VARCHAR(50),
    battle VARCHAR(100),
    result VARCHAR(20),
    PRIMARY KEY (ship, battle)
);
""")

# 2. Insert sample data
cur.execute("""
INSERT INTO Classes VALUES
('Bismarck', 'bb', 'Germany', 8, 15, 42000),
('Iowa', 'bb', 'USA', 9, 16, 46000),
('Kongo', 'bc', 'Japan', 8, 14, 32000),
('North Carolina', 'bb', 'USA', 9, 16, 37000),
('Renown', 'bc', 'Gt. Britain', 6, 15, 32000),
('Revenge', 'bb', 'Gt. Britain', 8, 15, 29000),
('Tennessee', 'bb', 'USA', 12, 14, 32000),
('Yamato', 'bb', 'Japan', 9, 18, 65000);
""")

cur.execute("""
INSERT INTO Battles VALUES
('Denmark Strait', '5/24-27/41'),
('Guadalcanal', '11/15/42'),
('North Cape', '12/26/43'),
('Surigao Strait', '10/25/44');
""")

cur.execute("""
INSERT INTO Outcomes VALUES
('Arizona', 'Pearl Harbor', 'sunk'),
('Bismarck', 'Denmark Strait', 'sunk'),
('California', 'Surigao Strait', 'ok'),
('Duke of York', 'North Cape', 'ok'),
('Fuso', 'Surigao Strait', 'sunk'),
('Hood', 'Denmark Strait', 'sunk'),
('King George V', 'Denmark Strait', 'ok'),
('Kirishima', 'Guadalcanal', 'sunk'),
('Prince of Wales', 'Denmark Strait', 'damaged'),
('Rodney', 'Denmark Strait', 'ok'),
('Scharnhorst', 'North Cape', 'sunk'),
('South Dakota', 'Guadalcanal', 'damaged'),
('Tennessee', 'Surigao Strait', 'ok'),
('Washington', 'Guadalcanal', 'ok'),
('West Virginia', 'Surigao Strait', 'ok'),
('Yamashiro', 'Surigao Strait', 'sunk');
""")

cur.execute("""
INSERT INTO Ships VALUES
('California', 'Tennessee', 1921),
('Haruna', 'Kongo', 1915),
('Hiei', 'Kongo', 1914),
('Iowa', 'Iowa', 1943),
('Kirishima', 'Kongo', 1915),
('Kongo', 'Kongo', 1913),
('Missouri', 'Iowa', 1944),
('Musashi', 'Yamato', 1942),
('New Jersey', 'Iowa', 1943),
('North Carolina', 'North Carolina', 1941),
('Ramillies', 'Revenge', 1917),
('Repulse', 'Renown', 1916),
('Resolution', 'Revenge', 1916),
('Revenge', 'Revenge', 1916),
('Royal Oak', 'Revenge', 1916),
('Royal Sovereign', 'Revenge', 1916),
('Renown', 'Renown', 1916),
('Washington', 'North Carolina', 1941),
('Wisconsin', 'Iowa', 1944),
('Yamato', 'Yamato', 1941);
""")

# Commit changes
conn.commit()

print("Tables created and sample data inserted into Neon DB.")

cur.close()
# conn.close()


Tables created and sample data inserted into Neon DB.


### **Ships launched before 1921**

In [20]:
query = """
SELECT name, launched
FROM Ships
WHERE launched < 1921;
"""

df = pd.read_sql_query(query, conn)
df


/tmp/ipython-input-2818020942.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,name,launched
0,Haruna,1915
1,Hiei,1914
2,Kirishima,1915
3,Kongo,1913
4,Ramillies,1917
5,Repulse,1916
6,Resolution,1916
7,Revenge,1916
8,Royal Oak,1916
9,Royal Sovereign,1916


### **Ships with their country and displacement**

In [21]:
query = """
SELECT s.name, c.country, c.displacement
FROM Ships s
JOIN Classes c ON s.class = c.class;
"""

df = pd.read_sql_query(query, conn)
df


/tmp/ipython-input-368087446.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,name,country,displacement
0,California,USA,32000
1,Haruna,Japan,32000
2,Hiei,Japan,32000
3,Iowa,USA,46000
4,Kirishima,Japan,32000
5,Kongo,Japan,32000
6,Missouri,USA,46000
7,Musashi,Japan,65000
8,New Jersey,USA,46000
9,North Carolina,USA,37000


### **List the ships engaged in the Battle of Guadalcanal with their displacement and number of guns.**


In [25]:
query = """
SELECT s.name, c.displacement, c.numGuns
FROM Outcomes o
JOIN Ships s ON o.ship = s.name
JOIN Classes c ON s.class = c.class
JOIN Battles b ON o.battle = b.name
WHERE b.name = 'Guadalcanal';
"""

df = pd.read_sql_query(query, conn)
df


/tmp/ipython-input-615365720.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,name,displacement,numguns
0,Kirishima,32000,8
1,Washington,37000,9


In [26]:
conn.close()